## Taking the unit prices of BTC and ETH with RestAPI and analyzing them on candlestick charts

In [1]:
from pycoingecko import CoinGeckoAPI
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

In [2]:
cg=CoinGeckoAPI()
#RestAPI module

### Data Prep

In [3]:
def candlestick_dataPreper(coin_name):
    
    try:
        
        coin_data=cg.get_coin_market_chart_by_id(id=coin_name,vs_currency="usd",days=30)

        coin_price_data=coin_data["prices"]

        dataCoin=pd.DataFrame(coin_price_data,columns=["TimeStamp","Price"])

        dataCoin["Date"]=pd.to_datetime(dataCoin["TimeStamp"],unit="ms")

        candlestick_dataCoin=dataCoin.groupby(dataCoin.Date.dt.date).agg({"Price":["min","max","first","last"]})
        
        return candlestick_dataCoin
        
    except:
        
        print("Please enter the coin name correctly..")
        
#For the generic coding of the collecting the data in the pd.DataFrame

In [4]:
candlestick_dataBTC=candlestick_dataPreper("bitcoin")
candlestick_dataETH=candlestick_dataPreper("ethereum")

### Data Visualization

In [16]:
def figDrawer(candlestick_data,coin_name,vs_currency):
    
    fig=go.Figure(data=[go.Candlestick(x=candlestick_data.index,
                    open=candlestick_data["Price"]["first"],
                    high=candlestick_data["Price"]["max"],
                    low=candlestick_data["Price"]["min"],
                    close=candlestick_data["Price"]["last"])
                    ])
    
    fig.update_layout(xaxis_rangeslider_visible=False,xaxis_title="Date",yaxis_title="Price ({})".format(vs_currency)
                  ,title="{} Candlestick Chart Over {} Days".format(coin_name,candlestick_dataBTC.index.shape[0]))
    
    
    return fig

#For the generic coding of the visualize the data

In [18]:
fig=figDrawer(candlestick_dataETH,"ETH","USD")
pio.write_html(fig, file='stock1_figure.html', auto_open=True)

In [19]:
figDrawer(candlestick_dataBTC,"BTC","USD")
pio.write_html(fig, file='stock2_figure.html', auto_open=True)